# Predictive analytics challenge
I've chosen time series (forecast) and anomaly detection for my 2 projects.


In [ ]:
import sqldf
import pandas as pd
import geopandas as gpd
import sklearn as sk
import helper.functions as hf
from datetime import datetime
import holidays

In [ ]:
# make logger
start = datetime.now()
logger = hf.make_logger('4-predictive_analytic_tasks_explore')

In [ ]:
# load data
col_types = {
    'notification_number':str,
    'reference_number':str
    }
date_cols = ['creation_timestamp','completion_timestamp']
df = pd.read_csv('data/raw/sr_hex.csv', parse_dates=date_cols,dtype=col_types)

# Data preparations
Calendar dates are often usefull added info.
We can get year, month and week (needed for aggregation) but we can also pull public holidays.
So we can try and make some values here for each of these.

For forecasts the 0 hexes might not be useful. They aren't a clustered area with a shared pattern.
We'll remove these

In [ ]:
# date calculations
df = df[df['h3_level8_index'] != '0']
df['create_yrweek'] = df['creation_timestamp'].dt.year.astype(str) + df['creation_timestamp'].dt.isocalendar().week.astype(str).map("{:0>2}".format)
df['create_date'] = pd.to_datetime(df['creation_timestamp'].dt.date)
df['date_diff'] = (df['completion_timestamp'] - df['creation_timestamp']).dt.days

In [ ]:
print(df['create_yrweek'].max(), df['create_yrweek'].min())

In [ ]:
# find dates for calendar years based on date range for df
sa_calendar = holidays.SouthAfrica(years=[2020,2021,2022])
sa_calendar = pd.DataFrame(sa_calendar.items(), columns=['date', 'desc'])
sa_calendar['date'] = pd.to_datetime(sa_calendar['date'])

In [ ]:
# Let's join this info to the data
df['create_pubday'] = df.merge(sa_calendar, how='left', left_on='create_date', right_on='date')['desc']

In [ ]:
# Let's take the cols we need for forecast only
df_f = df[['h3_level8_index','create_yrweek','create_date','create_pubday']]

query = '''
select h3_level8_index,
       create_yrweek,
       count(distinct create_pubday) pubday,
       count(*) req
from df_f
group by h3_level8_index, create_yrweek
order by h3_level8_index, create_yrweek
'''

df_f = sqldf.run(query)

In [ ]:
# We might have to fill in missing weeks
fill_df = pd.DataFrame({'create_yrweek': [str(x) for x in range(202001,202054)], 'req':0})
sa_calendar['yrweek'] = sa_calendar['date'].dt.year.astype(str) + sa_calendar['date'].dt.isocalendar().week.astype(str).map("{:0>2}".format)
sa_tmp = sa_calendar[['yrweek','desc']].groupby('yrweek').agg({'desc':'nunique'})
fill_df['pubday'] = df.merge(sa_tmp, how='left', left_on='create_yrweek', right_on='yrweek')['desc']
fill_df['pubday'] = fill_df['pubday'].fillna(0)

In [ ]:
# We can map the fill_df values now onto our forcast set
df_f2 = []
g = df_f.groupby('h3_level8_index')
for name, data in g:
    fill_df['h3_level8_index'] = name
    df_f2.append(pd.concat([data,fill_df[~fill_df['create_yrweek'].isin(data['create_yrweek'])]]))

df_f2 = pd.concat(df_f2)

query = '''
select h3_level8_index,
       create_yrweek,
       pubday,
       req
from df_f2
order by h3_level8_index, create_yrweek
'''

df_f = sqldf.run(query)
del df_f2

# Forecasting model and prep
We'll use a simple gradient boosting machine for this
* Benefits are insensitivity to data transformations
* Also we can use multivariate approaches

In [ ]:
# Build model using a loop
from xgboost import XGBRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregMultiOutput import ForecasterAutoregMultiOutput
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster

g = df_f.groupby('h3_level8_index')
pred_df = []

exog_variables = ['pubday']
end_train = 38
end_validation = 43
for name, data in g:
    data = data.reset_index(drop=True)
    data_train = data.loc[: end_train, :]
    data_val   = data.loc[end_train:end_validation, :]
    data_test  = data.loc[end_validation:, :]
    
    forecaster = ForecasterAutoreg(
                regressor = XGBRegressor(),
                lags = 4
                )
    param_grid = {
        'n_estimators': [100, 500],
        'max_depth': [3, 5, 10],
        'learning_rate': [0.01, 0.1]
        }

    # Lags used as predictors
    lags_grid = [4, [1, 2, 3, 4]]

    results_grid = grid_search_forecaster(
                            forecaster         = forecaster,
                            y                  = data.loc[:end_validation, 'req'],
                            exog               = data.loc[:end_validation, exog_variables],
                            param_grid         = param_grid,
                            #lags_grid          = lags_grid,
                            steps              = 36,
                            refit              = False,
                            metric             = 'mean_squared_error',
                            initial_train_size = int(len(data_train)),
                            fixed_train_size   = False,
                            return_best        = True,
                            verbose            = False
                            )
    # Backtesting
    metric, predictions = backtesting_forecaster(
        forecaster         = forecaster,
        y                  = data['req'],
        exog               = data[exog_variables],
        initial_train_size = len(data.loc[:end_validation]),
        fixed_train_size   = False,
        steps              = 36,
        refit              = False,
        metric             = 'mean_squared_error',
        verbose            = False
        )
    pred = predictions[-4:].reset_index(drop=True)
    pred['hex'] = name
    pred['backtest_error'] = metric
    pred_df.append(pred)
    del forecaster

In [ ]:
pred_df = pd.concat(pred_df)
pred_df.to_csv('4-predictive_analytic_tasks_forecast.csv', index=None)